## NLP_Assignment_6
1. What are Vanilla autoencoders
2. What are Sparse autoencoders
3. What are Denoising autoencoders
4. What are Convolutional autoencoders
5. What are Stacked autoencoders
6. Explain how to generate sentences using LSTM autoencoders
7. Explain Extractive summarization
8. Explain Abstractive summarization
9. Explain Beam search
10. Explain Length normalization
11. Explain Coverage normalization
12. Explain ROUGE metric evaluation

In [2]:
'''Ans 1:- Vanilla autoencoders are a type of neural network used for
unsupervised learning and dimensionality reduction. They consist of an
encoder that compresses input data into a lower-dimensional
representation and a decoder that attempts to reconstruct the original
input from this representation. 

In this example, the encoder compresses the input data,
and the decoder tries to reconstruct it, with the aim of
learning a meaningful representation in the bottleneck layer
(encoded). This is a basic illustration of a vanilla autoencoder,
which can be extended for various applications like data
denoising and feature extraction.

'encoded_data' contains the encoded representations,
and 'decoded_data' contains the reconstructed data.
'''

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# the dimensionscfor 28x28 pixel images
input_dim = 784
encoding_dim = 128

# encoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)

# decoder
decoded = Dense(input_dim, activation='sigmoid')(encoded)

# autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

num_samples = 1000
X_train = np.random.rand(num_samples, input_dim)

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=10, batch_size=32)

encoder_model = Model(input_layer, encoded)
encoded_data = encoder_model.predict(X_train)

decoded_data = autoencoder.predict(X_train)

Epoch 1/10
32/32 [==============================] - 1s 3ms/step - loss: 0.0834
Epoch 2/10
32/32 [==============================] - 0s 3ms/step - loss: 0.0832
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 0.0831
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 0.0831
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 0.0831
Epoch 6/10
32/32 [==============================] - 0s 3ms/step - loss: 0.0830
Epoch 7/10
32/32 [==============================] - 0s 3ms/step - loss: 0.0830
Epoch 8/10
32/32 [==============================] - 0s 3ms/step - loss: 0.0829
Epoch 9/10
32/32 [==============================] - 0s 3ms/step - loss: 0.0829
Epoch 10/10
32/32 [==============================] - 0s 1ms/step


In [3]:
'''Ans 2:- Sparse autoencoders are a variant of autoencoders that
encourage a sparse representation in the encoder's hidden layer.
They use techniques like L1 regularization to limit the number
of active neurons.This code defines a sparse autoencoder
using the Keras library, trains it on the MNIST dataset, and can
be used to extract a sparse representation ('encoded') of the
input data.'''

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist

# Load a dataset (e.g., MNIST)
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Define a sparse autoencoder
input_layer = Input(shape=(784,))
encoded = Dense(128, activation='relu', activity_regularizer=tf.keras.regularizers.l1(1e-5))(input_layer)
decoded = Dense(784, activation='sigmoid')(encoded)
autoencoder = Model(input_layer, decoded)

# Compile and train the model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train, epochs=50, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

11490434/11490434 [==============================] - 4s 0us/step
Epoch 1/50
235/235 [==============================] - 3s 9ms/step - loss: 0.2155 - val_loss: 0.1375
Epoch 2/50
235/235 [==============================] - 2s 8ms/step - loss: 0.1207 - val_loss: 0.1059
Epoch 3/50
235/235 [==============================] - 2s 7ms/step - loss: 0.0995 - val_loss: 0.0922
Epoch 4/50
235/235 [==============================] - 2s 7ms/step - loss: 0.0891 - val_loss: 0.0847
Epoch 5/50
235/235 [==============================] - 2s 7ms/step - loss: 0.0832 - val_loss: 0.0803
Epoch 6/50
235/235 [==============================] - 2s 8ms/step - loss: 0.0796 - val_loss: 0.0776
Epoch 7/50
235/235 [==============================] - 2s 7ms/step - loss: 0.0772 - val_loss: 0.0758
Epoch 8/50
235/235 [==============================] - 2s 8ms/step - loss: 0.0756 - val_loss: 0.0745
Epoch 9/50
235/235 [==============================] - 2s 7ms/step - loss: 0.0744 - val_loss: 0.0735
Epoch 10/50
235/235 [==============

In [4]:
'''Ans 3:- Denoising autoencoders are a type of autoencoder designed
to learn robust feature representations by training on noisy
data. They take noisy inputs and aim to reconstruct the
original, clean data. This code demonstrates how to create and train
a denoising autoencoder on noisy MNIST data, and 'encoded'
contains the denoised representations of the input images.
After training, we can use 'encoded' as a denoised representation.'''

import numpy as np
from tensorflow.keras.layers import Input, Dense, GaussianNoise
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Create noisy data
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

# Define a denoising autoencoder
input_layer = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_layer)
decoded = Dense(784, activation='sigmoid')(encoded)
autoencoder = Model(input_layer, decoded)

# Compile and train the model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train_noisy, x_train, epochs=10, batch_size=256, shuffle=True, validation_data=(x_test_noisy, x_test))

Epoch 1/10
235/235 [==============================] - 2s 8ms/step - loss: 0.2301 - val_loss: 0.1558
Epoch 2/10
235/235 [==============================] - 2s 7ms/step - loss: 0.1416 - val_loss: 0.1295
Epoch 3/10
235/235 [==============================] - 2s 7ms/step - loss: 0.1244 - val_loss: 0.1194
Epoch 4/10
235/235 [==============================] - 2s 7ms/step - loss: 0.1170 - val_loss: 0.1151
Epoch 5/10
235/235 [==============================] - 2s 7ms/step - loss: 0.1132 - val_loss: 0.1123
Epoch 6/10
235/235 [==============================] - 2s 7ms/step - loss: 0.1109 - val_loss: 0.1108
Epoch 7/10
235/235 [==============================] - 2s 7ms/step - loss: 0.1095 - val_loss: 0.1099
Epoch 8/10
235/235 [==============================] - 2s 7ms/step - loss: 0.1084 - val_loss: 0.1090
Epoch 9/10
235/235 [==============================] - 2s 7ms/step - loss: 0.1076 - val_loss: 0.1084
Epoch 10/10
235/235 [==============================] - 2s 7ms/step - loss: 0.1070 - val_loss: 0.1080

In [5]:
'''Ans 4:- Convolutional autoencoders are a type of autoencoder that
uses convolutional neural networks (CNNs) for feature
extraction and reconstruction of images. They are particularly suited
for tasks like image denoising and compression. In these
autoencoders, convolutional layers in the encoder capture hierarchical
image features, while transposed convolutional layers in the
decoder reconstruct the original image. here is an example using
Keras to create a convolutional autoencoder for image
reconstruction:'''

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import numpy as np

# Load and preprocess MNIST data
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Define a convolutional autoencoder
input_layer = Input(shape=(28, 28, 1))
encoded = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
encoded = MaxPooling2D((2, 2), padding='same')(encoded)
decoded = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(decoded)
autoencoder = Model(input_layer, decoded)

# Compile and train the model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train, epochs=10, batch_size=128, shuffle=True, validation_data=(x_test, x_test))

Epoch 1/10
469/469 [==============================] - 64s 136ms/step - loss: 0.1067 - val_loss: 0.0660
Epoch 2/10
469/469 [==============================] - 63s 134ms/step - loss: 0.0654 - val_loss: 0.0643
Epoch 3/10
469/469 [==============================] - 65s 138ms/step - loss: 0.0642 - val_loss: 0.0635
Epoch 4/10
469/469 [==============================] - 67s 142ms/step - loss: 0.0634 - val_loss: 0.0628
Epoch 5/10
469/469 [==============================] - 65s 139ms/step - loss: 0.0629 - val_loss: 0.0624
Epoch 6/10
469/469 [==============================] - 68s 146ms/step - loss: 0.0626 - val_loss: 0.0621
Epoch 7/10
469/469 [==============================] - 64s 136ms/step - loss: 0.0623 - val_loss: 0.0618
Epoch 8/10
469/469 [==============================] - 65s 138ms/step - loss: 0.0621 - val_loss: 0.0616
Epoch 9/10
469/469 [==============================] - 65s 138ms/step - loss: 0.0619 - val_loss: 0.0614
Epoch 10/10
469/469 [==============================] - 63s 134ms/step - l

In [ ]:
'''Ans 5:- Stacked autoencoders are a type of neural network
architecture composed of multiple layers of autoencoders. Each layer
learns to capture progressively higher-level features from the
input data. Stacking multiple autoencoders allows for more
complex feature extraction and dimensionality reduction. Stacked
autoencoders are often used for unsupervised pretraining of deep neural
networks, as they help initialize weights and improve the network's
ability to learn intricate patterns in the data.'''

In [ ]:
'''Ans 6:- To generate sentences using LSTM autoencoders, follow
these steps:- 

Data Preparation: Prepare a dataset of sentences,
tokenize them, and convert them into numerical sequences. 

Architecture: Design an LSTM autoencoder. The encoder LSTM compresses
input sequences into a fixed-size representation, while the
decoder LSTM generates sentences from this representation. 

Training: Train the autoencoder using the input sentences as both
input and target. Minimize the reconstruction error. 

Generation: To generate sentences, input a seed sequence to the
encoder and decode it using the decoder LSTM. Sample words at each
time step based on predicted probabilities.  

Repeat: Iterate the generation process, appending each generated
word to the seed sequence, until the desired sentence length or an end
token is reached.  By using LSTM autoencoders, the model learns
to capture the sequential dependencies in the data, allowing
it to generate coherent sentences based on the input seed
sequence.'''

In [ ]:
'''Ans 7:- Extractive summarization is a text summarization
technique where the summary is created by selecting and extracting
sentences or phrases directly from the original document. It does
not generate new sentences but rather identifies the most
relevant and informative content based on criteria like sentence
importance, coherence, and relevance scores. Extractive summarization
methods often employ natural language processing and machine
learning algorithms to rank and select sentences that best
represent the key information in the source text, resulting in a
concise summary that preserves the original content's structure
and context.'''

In [ ]:
'''Ans 8:- Abstractive summarization is a text summarization
technique that generates concise and coherent summaries by
interpreting and rephrasing the source content in a more human-like
manner. It goes beyond extraction and can create novel sentences
to convey the core ideas. For example, given the input "The
cat sat on the mat," an abstractive summarization model might
generate "A cat rested on the soft mat." It involves natural
language generation techniques and often relies on deep learning
models like transformers to achieve this.'''

In [ ]:
'''Ans 9:- Beam search is a search algorithm used in various natural
language processing tasks, including machine translation and text
generation. It explores multiple possible sequences of words during
decoding. Instead of selecting just the most likely word at each
step, it maintains a "beam" of the top-K candidates. This allows
for a broader exploration of possibilities and improves the
quality of generated text by considering multiple potential
continuations and selecting the best ones based on predefined criteria.'''

In [ ]:
'''Ans 10:- Length normalization is a technique used in natural
language processing to address bias in sequence generation models
like beam search. It involves adjusting the model's scoring or
ranking of generated sequences based on their length. Longer
sequences tend to have lower probabilities, so length normalization
ensures fairness by dividing the model's score by the length of
the sequence, penalizing overly short or long outputs. This
helps produce summaries or translations that are more balanced
and contextually appropriate.'''

In [ ]:
'''Ans 11:- Coverage normalization is a mechanism used in
sequence-to-sequence models for tasks like machine translation and text
summarization. It keeps track of how often each source input has been
attended to during the decoding process. This helps mitigate issues
like repeated words in the generated output. By encouraging the
model to attend to uncovered source parts, it promotes more
accurate and coherent generation of target sequences by reducing
redundancy and improving coverage of the input.'''

In [ ]:
'''Ans 12:- ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
is a metric used for automatic evaluation of the quality of
machine-generated text, particularly in tasks like text summarization or
machine translation. It measures the similarity between the
generated text and reference text using various measures, including
precision, recall, and F1-score, for n-grams (word sequences). ROUGE
helps assess the effectiveness of algorithms in producing
summaries or translations that match human-written references.'''